<a href="https://colab.research.google.com/github/viktorasb/Neural-Network-using-Tensorflow-keras-and-SVHN-Dataset/blob/master/keras_sklearn_class_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==0.20.3

     |████████████████████████████████| 5.4MB 2.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l2
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

file = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/' \
       'pima-indians-diabetes.data.csv'

# Load csv data from file to data using pandas
data = pd.read_csv(file, names=['pregnancies', 'glucose', 'diastolic', 'triceps', 'insulin',
                                'bmi', 'dpf', 'age', 'diabetes'])

# Process data
data.head()
x = data.drop(columns=['diabetes'])
y = data['diabetes']

# Split into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# define a sequential model
model = Sequential()
# 1st hidden layer
model.add(Dense(100, activation='relu', input_dim=8, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
# 2nd hidden layer
model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(1, activation='sigmoid'))
# Compilation with weighted metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], 
                         weighted_metrics=['accuracy'])

# Calculate validation _sample_weights_ based on the class distribution of train labels and 
# apply it to test labels using Sklearn
cls_weights = class_weight.compute_class_weight('balanced', np.unique(y_train._values), 
                                                y_train._values)
cls_weight_dict = {0: cls_weights[0], 1: cls_weights[1]}
val_sample_weights = class_weight.compute_sample_weight(cls_weight_dict, y_test._values)

# Train model
model_output = model.fit(x_train, y_train, epochs=500, batch_size=32, verbose=1,
                         validation_data=(x_test, y_test, val_sample_weights))

# Predict model
y_pred = model.predict(x_test, batch_size=32, verbose=1)

# Classify predictions based on threshold at 0.5
y_pred_binary = (y_pred > 0.5) * 1

# Sklearn metrics
sklearn_accuracy = accuracy_score(y_test, y_pred_binary)
sklearn_weighted_accuracy = accuracy_score(y_test, y_pred_binary, 
                                           sample_weight=val_sample_weights)

# metric_list has 3 entries: [0] val_loss weighted by val_sample_weights, [1] val_accuracy 
# [2] val_weighted_accuracy
metric_list = model.evaluate(x_test, y_test, batch_size=32, verbose=1, 
                             sample_weight=val_sample_weights)

print('sklearn_accuracy=%.3f' %sklearn_accuracy)
print('sklearn_weighted_accuracy=%.3f' %sklearn_weighted_accuracy)
print('keras_evaluate_accuracy=%.3f' %metric_list[1])
print('keras_evaluate_weighted_accuracy=%.3f' %metric_list[2])

Using TensorFlow backend.


Train on 691 samples, validate on 77 samples
Epoch 1/500
691/691 [==============================] - 1s 730us/step - loss: 7.5066 - accuracy: 0.5253 - accuracy_1: 0.5253 - val_loss: 3.8945 - val_accuracy: 0.6623 - val_accuracy_1: 0.5311
Epoch 2/500
691/691 [==============================] - 0s 64us/step - loss: 4.9952 - accuracy: 0.5702 - accuracy_1: 0.5702 - val_loss: 1.7595 - val_accuracy: 0.7403 - val_accuracy_1: 0.6950
Epoch 3/500
691/691 [==============================] - 0s 58us/step - loss: 3.7788 - accuracy: 0.5962 - accuracy_1: 0.5962 - val_loss: 1.9776 - val_accuracy: 0.7013 - val_accuracy_1: 0.6389
Epoch 4/500
691/691 [==============================] - 0s 54us/step - loss: 3.3305 - accuracy: 0.5919 - accuracy_1: 0.5919 - val_loss: 1.5967 - val_accuracy: 0.5714 - val_accuracy_1: 0.6416
Epoch 5/500
691/691 [==============================] - 0s 57us/step - loss: 3.2148 - accuracy: 0.5384 - accuracy_1: 0.5384 - val_loss: 1.5928 - val_accuracy: 0.6494 - val_accuracy_1: 0.6848
Epoc